# Downloading tcg datas

### Imports & Setup the environment

In [1]:
# Made on python 3.12

import pandas as pd #library for data analysis
import numpy as np #library for scientific computing with Python
import matplotlib.pyplot as plt #library for creating static, animated, and interactive visualizations
from matplotlib.image import imread #library for reading and writing image files
import cv2 #Manipulate images and videos,
import os #manage files and directories
from PIL import Image ,ImageOps # Python Imaging Library adds image processing capabilities to your Python interpreter. 
from dotenv import load_dotenv #load environment variables from .env file
import math #mathematical functions
import json #library for working with JSON
import requests #library for making HTTP requests
import shutil #library for copying and moving files
import io
from random import randrange, choice
import time
from io import BytesIO


load_dotenv()


True

### Global variables

In [2]:
API_NINJA_API_KEY = os.getenv("API_NINJA_API_KEY")
TCG_POKEMON_API_KEY = os.getenv("TCG_POKEMON_API_KEY")
TCG_POKEMON_MAX_CARDS_EXISTING = 19501
TCG_POKEMON_MAX_POKEMON_PER_PAGE = 250
DATA_TCG_FOLDER_PATH = "data/tcg/"
WINDOWS_DATA_FOLDER="/mnt/e/code/ai/data/tcg/"
GENERATED_DATAS_V1_FOLDER_PATH = "data/generated_datas_v1/"
IS_WSL = True
if IS_WSL :
    GENERATED_DATAS_V1_FOLDER_PATH = WINDOWS_DATA_FOLDER + GENERATED_DATAS_V1_FOLDER_PATH
    DATA_TCG_FOLDER_PATH = WINDOWS_DATA_FOLDER + DATA_TCG_FOLDER_PATH

GENERATED_DATAS_V1_IMAGES_FOLDER_PATH = GENERATED_DATAS_V1_FOLDER_PATH + "images/"
GENERATED_DATAS_V1_JSON_FOLDER_PATH = GENERATED_DATAS_V1_FOLDER_PATH + "json/"


### Configuration

In [3]:
if not os.path.exists(GENERATED_DATAS_V1_FOLDER_PATH):
    os.makedirs(GENERATED_DATAS_V1_FOLDER_PATH)
    print(GENERATED_DATAS_V1_FOLDER_PATH +" folder created")

if not os.path.exists(GENERATED_DATAS_V1_IMAGES_FOLDER_PATH):
    os.makedirs(GENERATED_DATAS_V1_IMAGES_FOLDER_PATH)
    print(GENERATED_DATAS_V1_IMAGES_FOLDER_PATH +" folder created")

if not os.path.exists(GENERATED_DATAS_V1_JSON_FOLDER_PATH):
    os.makedirs(GENERATED_DATAS_V1_JSON_FOLDER_PATH)
    print(GENERATED_DATAS_V1_JSON_FOLDER_PATH +" folder created")

### Load a random image for the background

In [4]:

def load_random_image(**kwargs):
    RESIZE_SIZES = kwargs.get("RESIZE_SIZES")
    RGB_COLOR = kwargs.get("RGB_COLOR")
    if RGB_COLOR == None:
        RGB_COLOR = True
    response = requests.get("https://api.api-ninjas.com/v1/randomimage",headers={"X-Api-Key": API_NINJA_API_KEY, 'Accept': 'image/jpg'})
    image = Image.open(BytesIO(response.content))
    if RESIZE_SIZES:
        image = image.resize(RESIZE_SIZES)
    if RGB_COLOR == False:
        image = ImageOps.grayscale(image)
    return image




In [5]:
TCG_CARDS_IDS_LIST = os.listdir(DATA_TCG_FOLDER_PATH)

def load_tcg_data(**kwargs):
    card_id = kwargs.get("card_id")
    if card_id == None:
        card_id = choice(TCG_CARDS_IDS_LIST)
    card_id = card_id.replace(".json", "")

    with open(DATA_TCG_FOLDER_PATH + card_id + ".json") as f:
        card_data = json.load(f)
    return card_data

def load_tcg_image(tcg_data,**kwargs):
    RESIZE_SIZES = kwargs.get("RESIZE_SIZES")
    RGB_COLOR = kwargs.get("RGB_COLOR")
    if RGB_COLOR == None:
        RGB_COLOR = True
    response = requests.get(tcg_data["images"]["large"],headers={"X-Api-Key": TCG_POKEMON_API_KEY, 'Accept': 'image/jpg'})
    image = Image.open(BytesIO(response.content))
    if RESIZE_SIZES:
        image = image.resize(RESIZE_SIZES)
    if RGB_COLOR == False:
        image = ImageOps.grayscale(image)
    return image



In [6]:

def apply_skew_effect(image, top_left_skew=(0, 0), top_right_skew=(0, 0), bottom_left_skew=(0, 0), bottom_right_skew=(0, 0)):
    
    # Convertir PIL en OpenCV si nécessaire
    if hasattr(image, 'size'):  # C'est une image PIL
        # Convertir PIL en array numpy
        img_array = np.array(image)
        # Convertir RGB en BGR pour OpenCV
        if len(img_array.shape) == 3:
            img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        image = img_array
    
    # Maintenant on a un array OpenCV
    if len(image.shape) == 3:
        height, width, _ = image.shape
    else:
        height, width = image.shape

    # Points d'origine (coins de l'image rectangulaire)
    src_points = np.float32([
        [0, 0],           # top-left
        [width-1, 0],     # top-right
        [0, height-1],    # bottom-left
        [width-1, height-1] # bottom-right
    ])
    
    # Points de destination (avec déformation personnalisée)
    dst_points = np.float32([
        # Top-left: position originale + décalage (x, y)
        [0 + top_left_skew[0], 0 + top_left_skew[1]],
        # Top-right: position originale + décalage (x, y)
        [width-1 + top_right_skew[0], 0 + top_right_skew[1]],
        # Bottom-left: position originale + décalage (x, y)
        [0 + bottom_left_skew[0], height-1 + bottom_left_skew[1]],
        # Bottom-right: position originale + décalage (x, y)
        [width-1 + bottom_right_skew[0], height-1 + bottom_right_skew[1]]
    ])
    
    # Calculer la matrice de transformation perspective
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    
    # Appliquer la transformation
    skewed_image = cv2.warpPerspective(image, matrix, (width, height))
    
    # Préparer les coordonnées finales des angles
    final_coordinates = {
        'top_left': (int(dst_points[0][0]), int(dst_points[0][1])),
        'top_right': (int(dst_points[1][0]), int(dst_points[1][1])),
        'bottom_left': (int(dst_points[2][0]), int(dst_points[2][1])),
        'bottom_right': (int(dst_points[3][0]), int(dst_points[3][1]))
    }
    
    return skewed_image, final_coordinates


In [7]:
def generate_tcg_v1_sample(**kwargs):
    
    """
        Fonction qui génère une image de Pokémon TCG
        
        Paramètres :
            card_id : ID du Pokémon TCG à générer
            max_width_resolution : Largeur maximale de l'image
            max_height_resolution : Hauteur maximale de l'image
            min_width_resolution : Largeur minimale de l'image
            min_height_resolution : Hauteur minimale de l'image
            width_resolution : Largeur de l'image aléatoire
            height_resolution : Hauteur de l'image aléatoire
            top_left_skew : Décalement de la partie haut gauche du Pokémon
            top_right_skew : Décalement de la partie haut droite du Pokémon
            bottom_left_skew : Décalement de la partie bas gauche du Pokémon
            bottom_right_skew : Décalement de la partie bas droite du Pokémon
            x_skew_power : Puissance du décalage de la partie gauche du Pokémon
            y_skew_power : Puissance du décalage de la partie gauche du Pokémon
        
        
    """

    card_id = kwargs.get("card_id") or None
    tcg_datas = load_tcg_data(card_id=card_id)
    tcg_image = load_tcg_image(tcg_datas)

    max_width_resolution = kwargs.get("max_width_resolution") or 2100
    max_height_resolution = kwargs.get("max_height_resolution") or 2100
    min_width_resolution = kwargs.get("min_width_resolution") or tcg_image.width + 50
    min_height_resolution = kwargs.get("min_height_resolution") or tcg_image.height + 50

    width_resolution = kwargs.get("width_resolution") or randrange(min_width_resolution, max_width_resolution)
    height_resolution = kwargs.get("height_resolution") or randrange(min_height_resolution, max_height_resolution)

    
    x_skew_power = randrange(0,kwargs.get("x_skew_power") or 3)
    x_skew_offset = tcg_image.width / 10
    y_skew_power = randrange(0,kwargs.get("y_skew_power") or 3)
    y_skew_offset = tcg_image.height / 10

    random_image = load_random_image(RESIZE_SIZES=(width_resolution,height_resolution))
    top_left_skew = kwargs.get("top_left_skew") or (randrange(0,int( x_skew_offset * x_skew_power + 1)), randrange(0,int( y_skew_offset * y_skew_power+ 1)))
    top_right_skew = kwargs.get("top_right_skew") or (-randrange(0,int( x_skew_offset * x_skew_power+ 1)), randrange(0,int( y_skew_offset * y_skew_power+ 1)))
    bottom_left_skew = kwargs.get("bottom_left_skew") or (randrange(0,int( x_skew_offset * x_skew_power+ 1)), -randrange(0,int( y_skew_offset * y_skew_power+ 1)))
    bottom_right_skew = kwargs.get("bottom_right_skew") or (-randrange(0,int( x_skew_offset * x_skew_power+ 1)), -randrange(0,int( y_skew_offset * y_skew_power+ 1)))

    result1, coords1 = apply_skew_effect(
        tcg_image.copy(),
        top_left_skew=top_left_skew,    # Décaler vers la droite de 30px, vers le bas de 20px
        top_right_skew=top_right_skew,  # Décaler vers la gauche de 15px, vers le bas de 25px
        bottom_left_skew=bottom_left_skew, # Décaler vers la droite de 20px, vers le haut de 10px
        bottom_right_skew=bottom_right_skew # Décaler vers la gauche de 40px, vers le haut de 30px
    )

    offset_x = random_image.width - tcg_image.width
    offset_y = random_image.height - tcg_image.height

    random_offset_x = randrange(0,offset_x-1)
    random_offset_y = randrange(0,offset_y-1)

    tmp = cv2.cvtColor(result1, cv2.COLOR_BGR2GRAY)
    _,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
    b, g, r = cv2.split(result1)
    rgba = [b,g,r, alpha]
    dst = cv2.merge(rgba,4)
    pil_image = Image.fromarray(dst)
    random_image.paste(pil_image, (random_offset_x,random_offset_y),pil_image)

    top_left_bb = (round((random_offset_x + coords1['top_left'][0])/random_image.width,6), round((random_offset_y + coords1['top_left'][1])/random_image.height,6))
    top_right_bb = (round((random_offset_x + coords1['top_right'][0])/random_image.width,6), round((random_offset_y + coords1['top_right'][1])/random_image.height,6))
    bottom_left_bb = (round((random_offset_x + coords1['bottom_left'][0])/random_image.width,6), round((random_offset_y + coords1['bottom_left'][1])/random_image.height,6))
    bottom_right_bb = (round((random_offset_x + coords1['bottom_right'][0])/random_image.width,6), round((random_offset_y + coords1['bottom_right'][1])/random_image.height,6))
  
    filename = f"{tcg_datas["id"]}_{int(time.time())}"

    data = {
        "asset" :  { "name": f"{filename}.jpg", "size": { "width": width_resolution, "height": height_resolution } } ,
        "card_id": tcg_datas["id"],
        "regions": [{"points": [{"x": top_left_bb[0], "y":  top_left_bb[1]}, {"x": top_right_bb[0], "y": top_right_bb[1]}, {"x": bottom_left_bb[0], "y": bottom_left_bb[1]}, {"x": bottom_right_bb[0], "y": bottom_right_bb[1]}]}],
    }

    return data, random_image, filename



In [8]:
def save_tcg_v1_sample():
    try:
        data, image, filename = generate_tcg_v1_sample()
        json.dump(data, open(f"{GENERATED_DATAS_V1_JSON_FOLDER_PATH}{filename}.json", "w"))
        cv2.imwrite(f"{GENERATED_DATAS_V1_IMAGES_FOLDER_PATH}{filename}.jpg", np.array(image))
        print(f"{filename} Created")
    except:
        print("Er")

In [10]:
for i in range(10):
    save_tcg_v1_sample()

hgss3-43_1753193719 Created
pop5-5_1753193724 Created
sma-SV88_1753193725 Created
ex14-45_1753193726 Created
base3-45_1753193728 Created
xy10-51_1753193728 Created
rsv10pt5-163_1753193729 Created
sv10-88_1753193730 Created
ex11-33_1753193731 Created
swshp-SWSH186_1753193733 Created
